#### GeekBrains, факультет Искусственного интелекта
#### курс "Алгоритмы анализа данных" (06 окт 2022 - 08 ноя 2022)
#### Урок 3. Логистическая регрессия. Log Loss

### Домашнее задание студента Бакеева А.П.
Сдайте задание до: 21 окт., 00:00 MSK

__1.__ Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

__2.__ На данных из урока изучите влияние гиперпараметров на ошибку алгоритма.

__3.__ Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.

__4.__ Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

Загружаем необходимые библиотеки и функции

In [1]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

__Решение 1__

In [2]:
# создадим массив для тестирования функций, которые будут выкалывать границы отрезка ]0, 1[
a = np.array([-0.1, 0, 0.0001, 0.2, 0.8, 0.9, 1, 1.1])

Загоним значения массива в диапазон ]0, 1[

In [3]:
# первый вариант, когда мы задаем минимальное и максимальное значения по краям отрезка
a_cleared = np.clip(a, 1e-12, 0.999999999999)
a_cleared

array([1.e-12, 1.e-12, 1.e-04, 2.e-01, 8.e-01, 9.e-01, 1.e+00, 1.e+00])

In [4]:
# второй вариант, когда мы сравниваем каждое значение 
# и если оно не проходит проверку, меняем его на нужное значение 
a_cleared = np.where(a > 0 , a, 0.01)
a_cleared = np.where(a_cleared < 1 , a_cleared, 0.999)
a_cleared

array([1.00e-02, 1.00e-02, 1.00e-04, 2.00e-01, 8.00e-01, 9.00e-01,
       9.99e-01, 9.99e-01])

теперь используем np.clip для защиты calc_logloss от некорректных данных

In [5]:
# функция из лекции, которая сломается
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [6]:
# функция, которую мы защитим от поломки
def calc_logloss_save(y, y_pred):
    
    y_pred_cleared = np.clip(y_pred, 1e-12, 0.999999999999)
    
    err = - np.mean(y * np.log(y_pred_cleared) + (1.0 - y) * np.log(1.0 - y_pred_cleared))
    err = np.sum(err)
    return err

In [7]:
# Тестовый массив
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0]) # Здесь заданы недопустимые значения, которые должны сломать calc_logloss

In [8]:
calc_logloss(y1, y_pred1)

C:\Users\s2e\AppData\Local\Temp/ipykernel_1952/2202339170.py:3: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
C:\Users\s2e\AppData\Local\Temp/ipykernel_1952/2202339170.py:3: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

In [9]:
calc_logloss_save(y1, y_pred1)

9.999778782803785e-13

Вообще-то мне кажется правильнее отрабатывать эту ситуцию через поднятие ошибки.

Если кто-то передал в функцию "неправильные" данные, то ему надо сообщить об этом.
Я бы применил try exept

In [10]:
# функция, которую мы защитим от поломки
def calc_logloss_save_2(y, y_pred):
    
    try:        
        err = - np.mean(y * np.log(y_pred_cleared) + (1.0 - y) * np.log(1.0 - y_pred_cleared))
        err = np.sum(err)
        return err
    except Exception:
        print(f'! недопустимые значения y_min={min(y_pred)}, y_max={max(y_pred)}')

In [11]:
calc_logloss_save_2(y1, y_pred1)

! недопустимые значения y_min=0, y_max=1


__Прежде чем приступить к следующим задачам восстановим необходимые материалы из лекции для решения этих задач__

In [12]:
# Задача классификации - подходит преподователь студенту или нет 
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [13]:
# функция стандартизации
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [14]:
# Делаем стандартизацию второго и третьего признака, так как они по масштабу отличаются от остальных
X_st = X.copy()
X_st[1, :] = calc_std_feat(X[1, :])
X_st[2, :] = calc_std_feat(X[2, :])

In [15]:
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [-0.70710678, -0.70710678, -0.35355339,  0.70710678,  0.        ,
        -1.06066017,  0.70710678,  2.47487373, -0.70710678, -0.35355339],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [16]:
# считаем ошибку
def calc_logloss_save(y, y_pred):
    
    y_pred_cleared = np.clip(y_pred, 1e-12, 0.999999999999)
    
    err = - np.mean(y * np.log(y_pred_cleared) + (1.0 - y) * np.log(1.0 - y_pred_cleared))
    err = np.sum(err)
    return err

In [17]:
# считаем сигмоиду - вероятность принадлежности к нужному классу
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

Logistic Regression

In [18]:
#  считаем веса модели
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss_save(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 5) == 0:
            print(f'it = {i}, W = {W}, err = {err}')
    return W

In [19]:
W = eval_model(X_st, y, iterations=5000, alpha=1e-2)

it = 1000, W = [-0.64954789 -0.85447173  0.52700042  0.49463844], err = 0.573722134726751
it = 2000, W = [-1.10207094 -1.10987615  0.59785361  0.7458667 ], err = 0.5397189653857617
it = 3000, W = [-1.51307551 -1.28206321  0.60895831  0.9738354 ], err = 0.514585397828936
it = 4000, W = [-1.89136766 -1.4064791   0.58450618  1.18498333], err = 0.4941796387567555
it = 5000, W = [-2.24312694 -1.50061025  0.53929297  1.38262738], err = 0.47679438100433325


In [20]:
W

array([-2.24312694, -1.50061025,  0.53929297,  1.38262738])

In [21]:
#  делаем предсказание о принадлежности к нужному классу
def predict(w, X):
    
    m = X.shape[1]
    
    y_predicted = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X))
    
#     За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[1]):
        if (A[:,i] > 0.5): 
            y_predicted[:, i] = 1
        elif (A[:,i] <= 0.5):
            y_predicted[:, i] = 0
    
    return y_predicted

In [22]:
y_predicted = predict(W, X_st)
y_predicted

array([[0., 0., 1., 0., 1., 1., 1., 0., 0., 1.]])

In [23]:
# для сравнения
# y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [24]:
#  делаем функцию для простой оценку качества предсказания accuracy
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [25]:
accuracy(y, y_predicted)

0.8

Надо отметить, что, в этом случае, качество предсказания получилось не очень высокое, acuracy = 0.8

__Задача 2__

На данных из урока изучите влияние гиперпараметров на ошибку алгоритма.

Вычислим значение W_final и err для следующих значений скорости обучения (alpha) и количества итераций (n_iter):

alpha = 1e-4, 1e-3, 1e-2, 1e-1, 1, 10

n_iter = 1000, 3000, 5000, 7000

In [26]:
alphas = [1e-4, 1e-3, 1e-2, 1e-1, 1, 10]
n_iters = [1000, 3000, 5000, 7000]

Доработаем функцию

In [27]:
#  считаем веса модели и ошибку
def eval_model_err(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 5) == 0:
            print(i, W, err)
    return W, err

In [28]:
for alpha in alphas:
    print()
    for n_iter in n_iters:        
        W, err = eval_model_err(X_st, y, n_iter, alpha=alpha)
        print(f'it = {n_iter}, W_final = {W}, err = {err}, alpha = {alpha} \n')


200 [ 0.4885937  -0.14045206  0.64748356  1.51108896] 1.3590732709815043
400 [ 0.48050543 -0.14265109  0.6472584   1.4991862 ] 1.3484738884562517
600 [ 0.4724497  -0.14486145  0.64701293  1.4873221 ] 1.337943726623034
800 [ 0.46442684 -0.1470832   0.64674706  1.47549718] 1.3274833631119651
1000 [ 0.4564372  -0.14931638  0.64646072  1.46371198] 1.317093367541014
it = 1000, W_final = [ 0.4564372  -0.14931638  0.64646072  1.46371198], err = 1.317093367541014, alpha = 0.0001 

600 [ 0.4724497  -0.14486145  0.64701293  1.4873221 ] 1.337943726623034
1200 [ 0.44848113 -0.15156103  0.6461538   1.45196703] 1.3067743013003228
1800 [ 0.42481765 -0.1583643   0.64510889  1.41697904] 1.276248164758261
2400 [ 0.4014683  -0.16527216  0.64387653  1.38237269] 1.2463797706353283
3000 [ 0.37844189 -0.17228521  0.64245562  1.34816273] 1.2171828305203864
it = 3000, W_final = [ 0.37844189 -0.17228521  0.64245562  1.34816273], err = 1.2171828305203864, alpha = 0.0001 

1000 [ 0.4564372  -0.14931638  0.646460

4000 [-25.03100993  -5.68146595  -4.97404805  15.67614691] 0.17146209639191817
5000 [-27.84652415  -6.2472724   -5.61020643  17.47929458] 0.15953906829816733
it = 5000, W_final = [-27.84652415  -6.2472724   -5.61020643  17.47929458], err = 0.15953906829816733, alpha = 1 

1400 [-15.15348038  -3.72190551  -2.6916368    9.33027549] 0.22919600864608203
2800 [-21.06875864  -4.88855372  -4.06986095  13.13365372] 0.19120122172357773
4200 [-25.62458144  -5.80063947  -5.1085458   16.0565305 ] 0.1688148123338489
5600 [-29.37426994  -6.55465742  -5.95368805  18.45653136] 0.1536998070572177
7000 [-32.57058871  -7.19778991  -6.66935614  20.49857077] 0.1427412677437087
it = 7000, W_final = [-32.57058871  -7.19778991  -6.66935614  20.49857077], err = 0.1427412677437087, alpha = 1 


200 [-28.38320874  -8.6948285   -5.06581732  18.98346193] 0.3548435550560321
400 [-36.49842565  -8.61805837  -7.55008098  23.94604235] 0.23507506446806598
600 [-40.97432648  -9.05515403  -8.74090556  26.29224417] 0.13919

C:\Users\s2e\AppData\Local\Temp/ipykernel_1952/2202339170.py:3: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
C:\Users\s2e\AppData\Local\Temp/ipykernel_1952/2202339170.py:3: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


5000 [-77.08590799 -15.40497521 -16.98506308  48.72468621] nan
it = 5000, W_final = [-77.08590799 -15.40497521 -16.98506308  48.72468621], err = nan, alpha = 10 

1400 [-50.9983053  -10.8401349  -10.79002587  32.21985881] 0.10202587020284434
2800 [-62.97090599 -13.05037502 -13.55519868  39.80284738] 0.08663994427865715
4200 [-72.33875793 -14.64282588 -15.81116998  45.7254866 ] nan
5600 [-80.44669862 -15.92981435 -17.82621726  50.84748892] nan
7000 [-87.77564826 -17.04372533 -19.68147477  55.4758697 ] nan
it = 7000, W_final = [-87.77564826 -17.04372533 -19.68147477  55.4758697 ], err = nan, alpha = 10 



Видно, что с ростом alpha ошибка падает. Однако при росте alpha от одного до десяти появляется ошибка вычисления. 

Попробуем вручную подобрать максимальную alpha из этого диапазона [1,10] Будем задавать значения от 1 до 10 с шагом 1, пока не наткнемся на ошибку

In [29]:
W, err = eval_model_err(X_st, y, iterations=7000, alpha=4)
W, err

1400 [-29.65800893  -6.61175733  -6.01736832  18.63793954] 0.15268684619944617
2800 [-40.27931684  -8.74176398  -8.38716969  25.41122934] 0.12196358413588497
4200 [-47.6314536  -10.19009167 -10.03059382  30.08360708] 0.10745429720896446
5600 [-53.44539235 -11.30581035 -11.34610957  33.77135766] 0.098428586767986
7000 [-58.37658636 -12.22364844 -12.47991989  36.895179  ] 0.09195217490881206


(array([-58.37658636, -12.22364844, -12.47991989,  36.895179  ]),
 0.09195217490881206)

верхнее допустимое значение alpha получилось 4

посмотрим насколько качественным будет предсказание при таких параметрах

In [30]:
predict(W, X_st)

array([[0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]])

In [31]:
accuracy(y, predict(W, X_st))

1.0

acuracy = 1 

Предсказание совпало с исходными данными на 100%

__Параметры при которых ошибка минимальна (err=0.09195) будут: alpha=4, iteration=7000__

__Задача 3__

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model.

__Решение 3__

Создадим функцию calc_pred_proba

In [32]:
def calc_pred_proba(w, X):
    
    m = X.shape[1]
    
    y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    y_pred = sigmoid(np.dot(w.T, X))  
        
    return y_pred

In [33]:
calc_pred_proba(W, X_st)

array([[3.51424890e-01, 3.14047812e-03, 1.00000000e+00, 1.28390746e-09,
        9.60401903e-01, 1.77711330e-02, 1.00000000e+00, 1.30770604e-02,
        6.62416457e-01, 9.99999995e-01]])

__Задача 4__ 

Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

In [34]:
def calc_pred(w, X, P=0.5): # по умолчанию порог вероятности будет 0.5
    
    m = X.shape[1]
    
    y_predicted = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X))
    
#     За порог отнесения к тому или иному классу используем полученное значение P
    for i in range(A.shape[1]):
        if (A[:,i] > P): 
            y_predicted[:, i] = 1
        elif (A[:,i] <= P):
            y_predicted[:, i] = 0
    
    return y_predicted

Посчитаем, при пороге вероятности = 0,6

In [35]:
P = 0.6 # порог отнесения к классу 1
display(calc_pred(W, X_st, P), accuracy(y, calc_pred(W, X_st, P)))

array([[0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]])

1.0

Посмотрим что будет, если понизить порог

In [36]:
P = 0.3 # порог отнесения к классу 1
display(calc_pred(W, X_st, P), accuracy(y, calc_pred(W, X_st, P)))

array([[1., 0., 1., 0., 1., 0., 1., 0., 1., 1.]])

0.9